In [10]:
queries = {
    'Q2': '''
SELECT s_acctbal, s_name, n_name, p_partkey, p_mfgr
    , s_address, s_phone, s_comment
FROM part, supplier, partsupp, nation, region
WHERE p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 30
    AND p_type LIKE '%STEEL'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND ps_supplycost = (
        SELECT MIN(ps_supplycost)
        FROM partsupp, supplier, nation, region
        WHERE p_partkey = ps_partkey
            AND s_suppkey = ps_suppkey
            AND s_nationkey = n_nationkey
            AND n_regionkey = r_regionkey
            AND r_name = 'ASIA'
    )
ORDER BY s_acctbal DESC, n_name, s_name, p_partkey
LIMIT 100
    ''',
    'Q17': '''
SELECT sum(l_extendedprice) / 7.0 AS avg_yearly
FROM lineitem, part
WHERE p_partkey = l_partkey
    AND p_brand = 'Brand#44'
    AND p_container = 'WRAP PKG'
    AND l_quantity < (
        SELECT 0.2 * avg(l_quantity)
        FROM lineitem
        WHERE l_partkey = p_partkey
    )
    '''
}

In [39]:
import pymysql
import pymysql.cursors

class Connection:
    def __init__(self, host, port, user, password):
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.conn = None
    
    def connect(self, database):
        self.conn = pymysql.connect(host=self.host,
                                    port=self.port,
                                    user=self.user,
                                    passwd=self.password,
                                    db=database,
                                    charset='utf8')
    
    def select(self, sql):
        with self.conn.cursor() as cursor:
            cursor.execute(sql)
            result = cursor.fetchall()
        self.conn.commit()
        return str(result)
        
            
    def __del__(self):
        self.conn.close()

In [27]:
opt = Connection('127.0.0.1', 3306, 'test', '123456')
opt.connect('dbt3sf10')

opt.select("set global innodb_adaptive_hash_index = off;")
result_of_queries = {}

for name in queries.keys():
    result_of_queries[name] = {}
    sql = queries[name]
    print (sql)
    
    opt.select("set optimizer_switch='subquery_unnesting_with_window=on';")
  
    result = %timeit -o -n 1 -r 10 opt.select(sql)
    result_of_queries[name]['unnest'] = (result.average, result.worst, result.best)
    
    opt.select("set optimizer_switch='subquery_unnesting_with_window=off';")

    result = %timeit -o -n 1 -r 10 opt.select(sql)
    result_of_queries[name]['non-unnest'] = (result.average, result.worst, result.best)
    
    print("%s speedup is: %f." % (name, result_of_queries[name]['non-unnest'][0] / result_of_queries[name]['unnest'][0]))



SELECT s_acctbal, s_name, n_name, p_partkey, p_mfgr
    , s_address, s_phone, s_comment
FROM part, supplier, partsupp, nation, region
WHERE p_partkey = ps_partkey
    AND s_suppkey = ps_suppkey
    AND p_size = 30
    AND p_type LIKE '%STEEL'
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND ps_supplycost = (
        SELECT MIN(ps_supplycost)
        FROM partsupp, supplier, nation, region
        WHERE p_partkey = ps_partkey
            AND s_suppkey = ps_suppkey
            AND s_nationkey = n_nationkey
            AND n_regionkey = r_regionkey
            AND r_name = 'ASIA'
    )
ORDER BY s_acctbal DESC, n_name, s_name, p_partkey
LIMIT 100
    
1.2 s ± 30.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
1.84 s ± 32.4 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
Q2 speedup is: 1.535728.

SELECT sum(l_extendedprice) / 7.0 AS avg_yearly
FROM lineitem, part
WHERE p_partkey = l_partkey
    AND p_brand = 'Brand#44'
    

In [20]:
print(result_of_queries)

{'Q2': {'unnest': (1.2141587385998718, 1.232421185000021, 1.1703475929998604), 'non-unnest': (1.814908732799995, 1.8435294040000372, 1.780860814999869)}, 'Q17': {'unnest': (1.4744295098000293, 1.5379993710002964, 1.423272003999955), 'non-unnest': (6.9418928264001805, 7.01334007100013, 6.893790168000123)}}
